# Amazon Reviews Exploratory Data Analysis and Cleaning Script

For this project, I will only be using the 'amazon_reviews_us_Beauty_v1_00.tsv' data set. This file was downloaded individually on the AWS CLI using an API command provided by the link below. All files in this project will be stored in it's respective folders from my AWS S3 bucket. All scripts were produced using the Databricks Community Edition.

- **Link to Kaggle Data Set**: https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset
- **Link to Databricks Community Edition**: https://docs.databricks.com/en/getting-started/community-edition.html

**Note: This dataset is 22 GB. Please consider copying the API command that is available at the link to download individual files that you will be working with.**

# Import `os` and set up the environment variables for your AWS access/secret Keys
**Disclaimer: It is important that you do not share your AWS access/secret keys with anyone. Please double-check your script before posting/sharing your script to the public**

In [0]:
import os
# To work with Amazon S3 storage, set the following variables using your AWS Access Key and Secret Key
# Set the Region to where your files are stored in S3.
# Replace 'your-access-key' and 'your-secret-key' with your actual AWS access/secret keys
access_key = 'your-access-key'
secret_key = 'your-secret-key'
# Set the environment variables so boto3 can pick them up later
os.environ['AWS_ACCESS_KEY_ID'] = access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = secret_key
encoded_secret_key = secret_key.replace("/", "%2F")
# Note: You may need to change the aws_region depending on where your AWS S3 is located
aws_region = "us-east-2"
# Update the Spark options to work with our AWS Credentials
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region +
".amazonaws.com")


# Install all necessary libraries

In [0]:
 %pip install fsspec
 %pip install s3fs

# Import all necessary libraries

In [0]:
import os
import io
import pandas as pd
import s3fs
import boto3
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import HTML, display
from datetime import datetime
from pyspark.sql.functions import *


# Create File Path for your AWS S3 Bucket File(s)

In [0]:
# Set up the path to an Amazon reviews data stored on S3
# replace 'bucket-name-xx' with your actual bucket name. Replace 'xx' with your initials.
bucket = 'bucket-name-xx/landing/'
filename = 'amazon_reviews_us_Beauty_v1_00.tsv'
file_path = 's3a://' + bucket + filename

# Create a Spark Dataframe from the file from your AWS S3 Bucket

In [0]:
# This can take a long time if the file is big
# This one in particular takes about 1.10 minutes
sdf = spark.read.csv(file_path, sep='\t', header=True, inferSchema=True)

# Check the Data Types of your Spark DataFrame
- **This step is crucial for understanding how you will want to proceed to feature engineering.**
- **Here is how you should consider mapping out your data type conversion in preparation for feature engineering:**
- **String** --> String Indexer --> Encoder --> Vector Assembler
- **Long/Int** --> Encoder --> Vector Assembler
- **Float/Double** --> Vector Assembler

In [0]:
# Check the schema
sdf.printSchema()

# Count the Null/NaN values in your Spark DataFrame
- **This helps us understand how many records or columns will need to be cleaned/dropped/or back-filled (if-neccessary)**
- **We will also need to do this before we start converting our data types in order to avoid any errors in conversion.**

In [0]:
# Check to see the NULL/NaN values in your first 5 columns 
# Here, we are checking 5 columns at a time so that the visual output doesn't get too cluttered
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in
["marketplace", "customer_id", "review_id", "product_id", "product_parent"]] ).show()


In [0]:
# Check to see NULL/NaN values for the next 5 columns
# Note: You should see that product_category, star_rating, helpful_votes, and total_votes have 213-214 null values
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in
["product_title", "product_category", "star_rating", "helpful_votes", "total_votes"]] ).show()

In [0]:
# Check to see NULL/NaN values for the remaining 4 columns
# Note: We will check for Null values from review_date in another cell, since isnan cannot be executed with "date" data types
# Running isnan on a date data type will result in a data mismatch error
# You should see that vine, verified purchase, and review_headlines has 214-223 null/nan values
# You should also see that review_body has 916 null/nan values
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in
["vine", "verified_purchase", "review_headline", "review_body"]] ).show()

In [0]:
# Count only the null values for the review_date column and show the count
# You should see that review_date has 563 null values
null_count = sdf.agg(count(when(col("review_date").isNull(), "review_date"))).collect()[0][0]

print(f"Review Date Column Null Count: {null_count}")


# Get the Number of Records in the DataFrame

In [0]:
# We will compare this number to the count after cleaning for Null/NaN values to see how much was cleaned
# You should see that this DataFrame has 5,115,666 million records
sdf.count()

# Drop the Null/NaN records

In [0]:
# Drop some of the records where the certain columns are empty (Null/NaN)
sdf = sdf.na.drop(subset=["product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date"])

# Double-check to see if the columns still have Null/NaN values:

In [0]:
# You should see that the review_headline still have a Null/NaN value
sdf.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in
["product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body"]] ).show()

In [0]:
# Re-count only the null values for the review_date column and show the count
# You should see that review_date has 0 null values
null_count = sdf.agg(count(when(col("review_date").isNull(), "review_date"))).collect()[0][0]

print(f"Review Date Column Null Count: {null_count}")

# Drop any un-necessary columns

In [0]:
# Drop the un-necessary columns. Since the review_date is our most unique attribute, we can keep it in our Spark DataFrame.
# Later, we will see how we can split the review_date column into more aggregable columns!
sdf = sdf.drop("marketplace", "customer_id", "review_id", "product_id", "product_parent")

# Display schema
sdf.printSchema()

# Get rid of any non-ascii characters
**By removing non-ASCII characthers, we can standardize the text. Non-ASCII characters can contain special symbols, accented letters, and characters from various languages that may weaken our predictive modeling power later on.**

In [0]:
# Define a function to strip out any non-ascii characters
def ascii_only(mystring):
    if mystring:
        return mystring.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [0]:
# Turn this function into a User-Defined Function (UDF)
ascii_udf = udf(ascii_only)

In [0]:
# Clean up the review_body
sdf = sdf.withColumn("review_body", ascii_udf('review_body'))

# Display schema
sdf.printSchema()

In [0]:
# Check the length of the text in review_body and review_headline
sdf = sdf.withColumn("review_body_length", length(sdf.review_body))
sdf = sdf.withColumn("review_headline_length", length(sdf.review_headline))

sdf.printSchema()

# Check the statistics of the review body and review headline

In [0]:
# Some may have a minimum of 1, which may be an emoji that can't be encoded
sdf.select("review_body_length", "review_headline_length").summary().show()

In [0]:
# Filter by removing short reviews
sdf = sdf.where(sdf.review_body_length > 10)
sdf = sdf.where(sdf.review_headline_length > 10)

In [0]:
# Re-check the cleaned headline and body

print('review_headline: ')
sdf.select("review_headline").summary("count", "min",
"max").show()

print('review_body: ')
sdf.select("review_body").summary("count", "min",
"max").show()

# Convert data types if necessary

In [0]:
from pyspark.sql.functions import col

# Convert 'star_rating' column from string to integer using cast
sdf = sdf.withColumn("star_rating", col("star_rating").cast("int"))

# Print the schema after the conversion
sdf.printSchema()

# Create a few extra columns based on the date
**This will be useful for when you are running statistical analysis by year, month, or day**

In [0]:
from pyspark.sql.functions import year, month, date_format, when

# Create a few extra columns based on the date
sdf = sdf.withColumn("review_year", year(col("review_date")))
sdf = sdf.withColumn("review_month", month(col("review_date")))
sdf = sdf.withColumn("review_yearmonth", date_format(col("review_date"), "yyyy-MM"))   # Like 2023-01   2023-02 etc.
sdf = sdf.withColumn("review_dayofweek", date_format(col("review_date"), "E"))         # 'Monday' 'Tuesday' etc.
sdf = sdf.withColumn("review_weekend", when(sdf.review_dayofweek == 'Saturday',1.0).when(sdf.review_dayofweek == 'Sunday', 1.0).otherwise(0))

# Print the schema after the splitting the review_date column
sdf.printSchema()

# Get some statistics and graphs of your cleaned data

In [0]:
# Get some statistics on each of the columns (Warning: This can take a long time, unless you run it on the sample sdf)
sdf.summary().show()

In [0]:
# Take a sample of the data without replacement
sdf_sample = sdf.sample(False, 0.25)

In [0]:
# Look at statistics for some specific columns
# Note: Here we are only taking the statistics from a sample of the date
# To take statistics from the full data set, please replace 'sdf_sample' with 'sdf'
sdf_sample.select("star_rating", "helpful_votes", "total_votes").summary("count", "min",
"max", "mean").show()

In [0]:
# Look at the statistics for the Review headline and Review Body
sdf_sample.select("vine", "verified_purchase", "review_body").summary("count", "min", "max").show()

In [0]:
# Produce a bar chart that displays Number of Reviews by Review Date
import matplotlib.pyplot as plt
# Use groupby to get a count by date. Then convert to pandas dataframe
df = sdf.groupby("review_date").count().sort("review_date").toPandas()

# Matplotlib create a figure
fig = plt.figure(facecolor='white')
plt.bar(df['review_date'], df['count'])
# Set the x-axis label
plt.xlabel("Review Date")
# Set the y-axis label
plt.ylabel("Number of Reviews")
# Set the Title of the plot
plt.title("Number of Reviews by Date")
plt.xticks(rotation=90, ha='right')
fig.tight_layout()
# Save the figure
plt.savefig("review_count_by_date_matplotlib.png")
# If running in Jupyter Notebook call plt.show()
plt.show()


In [0]:
# Save the figure
from io import BytesIO

# Save the figure to a BytesIO object
img_data = BytesIO()
fig.savefig(img_data, format='png')
img_data.seek(0)

# Your S3 bucket details
# Replace 'bucket-name-xx' with your actual bucket name. Replace 'xx' with your initials.
bucket_name = 'bucket-name-xx'
s3_path = 'models/'

# Upload the figure to S3 using boto3
# Replace 'your-access-key' and 'your-secret-key' with your actual AWS access/secret keys
s3 = boto3.client('s3', aws_access_key_id='your-access-key', aws_secret_access_key='your-secret-key')
s3.upload_fileobj(img_data, bucket_name, s3_path)

# Close the plot
plt.close()

# Print statement
print(f"Figure saved to S3: {bucket_name}/{s3_path}")

In [0]:
# Produce a bar chart that displays Number of Star Ratings by Star Rating
import matplotlib.pyplot as plt
# Use groupby to get a count by rating. Then convert to pandas dataframe
df = sdf.groupby("star_rating").count().sort("star_rating").toPandas()

# Matplotlib create a figure
fig = plt.figure(facecolor='white')
plt.bar(df['star_rating'], df['count'])
# Set the x-axis label
plt.xlabel("Star Rating")
# Set the y-axis label
plt.ylabel("Number of Star_Ratings")
# Set the Title of the plot
plt.title("Number of Star Ratings by Star Rating")
plt.xticks(rotation=90, ha='right')
fig.tight_layout()
# Save the figure
plt.savefig("Number_of_Star_Rating_by_Star_Rating.png")
# If running in Jupyter Notebook call plt.show()
plt.show()

In [0]:
# Save the figure
from io import BytesIO

# Save the figure to a BytesIO object
img_data = BytesIO()
fig.savefig(img_data, format='png')
img_data.seek(0)

# Your S3 bucket details
# Replace 'bucket-name-xx' with your actual bucket name. Replace 'xx' with your initials.
bucket_name = 'bucket-name-xx'
s3_path = 'models/'

# Upload the figure to S3 using boto3
# Replace 'your-access-key' and 'your-secret-key' with your actual AWS access/secret keys
s3 = boto3.client('s3', aws_access_key_id='your-access-key', aws_secret_access_key='your-secret-key')
s3.upload_fileobj(img_data, bucket_name, s3_path)

# Close the plot
plt.close()

# Print statement
print(f"Figure saved to S3: {bucket_name}/{s3_path}")

In [0]:
# Produce a bar chart that displays Number of Reviews by Day of the Week
import matplotlib.pyplot as plt
# Use groupby to get a count of reviews by day of the week. Then convert to pandas dataframe
df = sdf.groupby("review_dayofweek").count().sort("review_dayofweek").toPandas()

# Matplotlib create a figure
fig = plt.figure(facecolor='white')
plt.bar(df['review_dayofweek'], df['count'])
# Set the x-axis label
plt.xlabel("Day of the Week")
# Set the y-axis label
plt.ylabel("Number of Reviews")
# Set the Title of the plot
plt.title("Number of Reviews by Day of the Week")
plt.xticks(rotation=90, ha='right')
fig.tight_layout()
# Save the figure
plt.savefig("Number_of_Reviews_by_DayofWeek.png")
# If running in Jupyter Notebook call plt.show()
plt.show()

In [0]:
# Save the figure
from io import BytesIO

# Save the figure to a BytesIO object
img_data = BytesIO()
fig.savefig(img_data, format='png')
img_data.seek(0)

# Your S3 bucket details
# Replace 'bucket-name-xx' with your actual bucket name. Replace 'xx' with your initials.
bucket_name = 'bucket-name-xx'
s3_path = 'models/'

# Upload the figure to S3 using boto3
# Replace 'your-access-key' and 'your-secret-key' with your actual AWS access/secret keys
s3 = boto3.client('s3', aws_access_key_id='your-access-key', aws_secret_access_key='your-secret-key')
s3.upload_fileobj(img_data, bucket_name, s3_path)

# Close the plot
plt.close()

# Print statement
print(f"Figure saved to S3: {bucket_name}/{s3_path}")

# Save your cleaned Spark DataFrame into your AWS S3 '/raw' folder as a Parquet File
**Parquet files tend to read into Spark DataFrames quicker. This will be useful later in our Feature Engineering and Modeling phase**

In [0]:
# Specify your S3 bucket and path
# Replace "bucket-name-xx" with your actual bucket name and replace 'xx' with your initials.
s3_bucket = "bucket-name-xx"
s3_path = "s3a://{}/raw/amazon_reviews_us_Beauty_v1_00_cleaned.parquet".format(s3_bucket)

# Save the DataFrame to S3 in Parquet format
# Note: "overwrite" will save over any previous files in the s3_path with the same file name.
sdf.write.parquet(s3_path, mode="overwrite")

# Display a success message
print("DataFrame saved to S3 successfully!: {}".format(s3_path))